In [0]:
import lxml.etree as etree
import urllib.request
import os.path
import re
import lxml.html
import csv
import signal
import requests

In [0]:
#открываем файл со ссылками. Всего 1475 ссылок, не считая errors - отдельный файл
with open('/content/drive/My Drive/ВКР/pisma_urs1.txt', 'r', encoding='utf-8') as file:
  urls = file.readlines()
  print(type(urls))
  print(len(urls))

In [0]:
#ЭТАП 1 - отсеиваем по заголовкам

ignore_head = ['лицо победы', 'из книги', 'из воспоминаний', 'дневник', 'из статьи', 'из приказа', 'из письма'] #список стоп-слов
ignore_links = [] #список для ссылок, которые игнорим
good_links = [] #список ссылок, которые обрабатываем + здесь отсеить по тэгам
errors = []
for url in urls:
  try:
    url = url.replace('\n', '')
    r = requests.get(url)
    page_source = r.text
    soup = BeautifulSoup(page_source, 'lxml')
    head = soup.find('strong').text
    lst = []
    if any(substr in head.lower() for substr in ignore_head):
      ignore_links.append(url)
    else:
      good_links.append(url)
  except:
    errors.append(url)

print('Хороших ссылок на 1 этапе: '+ str(len(good_links)))
print('Плохих ссылок на 1 этапе: '+ str(len(ignore_links)))
print('Ошибок на 1 этапе: '+ str(len(errors)))


#ЭТАП 2 получившийся список отсеиваем по тэгам, пополняем спиоск ignore, список хороших ссылок делаем новый

ignore_tag = ['воспоминания', 'дневник', 'газета', 'письмо в редакцию', 'письмо в сельсовет', 'немцы']
good_links2 = []
for url in good_links:
  try:
    url = url.replace('\n', '')
    r = requests.get(url)
    page_source = r.text
    soup = BeautifulSoup(page_source, 'lxml')
    full_text = soup.find('div', {'class':'flex_display flex_vbox'})
    sub_headings = []
    tags = []
    for sub_heading in full_text.find_all('p'):
      sub_headings.append(sub_heading.text)
      for prgrf in sub_headings:
        if prgrf == 'Tags:':
          tags = ' '.join(sub_headings[sub_headings.index(prgrf):])
          
    if any(substr in tags.lower() for substr in ignore_tag):
      ignore_links.append(url)
    else:
      good_links2.append(url)
  except AttributeError:
    errors_tag.append(url)

print('Хороших ссылок на 2 этапе: '+ str(len(good_links2)))
print('Плохих ссылок на 2 этапе: '+ str(len(ignore_links)))
print('Ошибок на 2 этапе: '+ str(len(errors)))

In [0]:
#тоже самое делаем со ссылками из файла с ошибками (316 ссылок)

with open('/content/drive/My Drive/ВКР/pisma_errors.txt') as e:
  errors = e.read()

errors = errors.split(sep='Error: ')
print(errors)

errors_list = []
for i in errors:
  q = i.split(sep=' : ')
  errors_list.append(q[0])

In [0]:
good_links = []
for url in errors_list:
  try:
    url = url.replace('\n', '')
    r = requests.get(url)
    page_source = r.text
    soup = BeautifulSoup(page_source, 'lxml')
    head = soup.find('strong').text
    lst = []
    if any(substr in head.lower() for substr in ignore_head):
      ignore_links.append(url)
    else:
      good_links.append(url)
  except:
    errors.append(url)

print('Хороших ссылок на 1 этапе: '+ str(len(good_links)))
print('Плохих ссылок на 1 этапе: '+ str(len(ignore_links)))
print('Ошибок на 1 этапе: '+ str(len(errors)))

ignore_tag = ['воспоминания', 'дневник', 'газета', 'письмо в редакцию', 'письмо в сельсовет', 'немцы']
for url in good_links:
  try:
    url = url.replace('\n', '')
    r = requests.get(url)
    page_source = r.text
    soup = BeautifulSoup(page_source, 'lxml')
    full_text = soup.find('div', {'class':'flex_display flex_vbox'})
    sub_headings = []
    tags = []
    for sub_heading in full_text.find_all('p'):
      sub_headings.append(sub_heading.text)
      for prgrf in sub_headings:
        if prgrf == 'Tags:':
          tags = ' '.join(sub_headings[sub_headings.index(prgrf):])
          
    if any(substr in tags.lower() for substr in ignore_tag):
      ignore_links.append(url)
    else:
      good_links2.append(url)
  except AttributeError:
    errors_tag.append(url)

In [0]:
#делим хорошие ссылки на три списка по количеству категорий
letters_one_cat = []
letters_sev_cat = []
letters_no_cat = []

#разбиваем все письма по категориям
for url in good_links2:
  r = requests.get(url)
  page_source = r.text
  soup = BeautifulSoup(page_source, 'lxml')
#создаем список категорий. Разделяем на две группы: с одной категорией, с несколькими категориями  
  category_list = []
  for i in soup.find_all('div', {'class': 'style-j15atrbplink'}):
    category_list.append(i.find('span').text)
  if len(category_list) == 1:
    letters_one_cat.append(url)
  if len(category_list) == 0:
    letters_no_cat.append(url)   #ВАЖНО!!!! Часть страниц, действительно, без категории!
  if len(category_list) > 1:
    letters_sev_cat.append(url) 

print('Одна категория:' + str(len(letters_one_cat))) 
print('Несколько категорий:' + str(len(letters_sev_cat))) 
print('Нет категорий:' + str(len(letters_no_cat))) 

In [0]:
#сохраняем все в файлики, чтобы не потерять

with open('pisma_good_links1.txt', 'a', encoding='utf-8') as file:
    for i in letters_one_cat:
        file.write("%s\n" % i)

with open('pisma_good_links2.txt', 'a', encoding='utf-8') as file:
    for i in letters_sev_cat:
        file.write("%s\n" % i)

with open('pisma_good_links0.txt', 'a', encoding='utf-8') as file:
    for i in letters_no_cat:
        file.write("%s\n" % i)

with open('pisma_ignore_links.txt', 'a', encoding='utf-8') as file:
    for i in ignore_links:
        file.write("%s\n" % i)

with open('pisma_error_links.txt', 'a', encoding='utf-8') as file:
    for i in errors:
        file.write("%s\n" % i)